In [80]:
import pandas as pd
from ensemble_boxes import *
import numpy as np
from pycocotools.coco import COCO

In [84]:
csvs = {
    'swinL_384' : '../Swin-Transformer-Object-Detection/work_dirs/swin/large_384_ms_50/submission.csv',
    #'swinL_384_pseudo' : 'swin_pseudo_384.csv',
    'swinL_224' : 'swin224.csv',
    'swin_balanced' : 'swin_balanced.csv',
    'swin_anchor' : 'swin_anchor_pseudo.csv',
    'swin_224_pseudo' : 'swin_pseudo.csv',
    'yolo5l6' : '../yolov5/submission_yolo5l6_pseudo_ms.csv',
    'yolo5x6' : '../yolov5/submission_yolo5x6_pseudo_ms.csv',
    'effd4' : 'submission_effd4_pseudo.csv',
    'effd5' : 'submission_effd5_pseudo.csv',
       }

In [18]:
submission_files = [f'../efficientdet-pytorch/submission_effdet_d4_ap_{imgsz}.csv' for imgsz in [512, 768, 1024, 1536, 2048]]
submission_df = [pd.read_csv(file) for file in submission_files]
weights = [1, 1, 1, 1, 1]

In [85]:
# ensemble csv files
#submission_files = ['swinL_384', 'swinL_384_pseudo', 'swinL_224', 'swin_224_pseudo', 'swin_anchor', 'swin_balanced']
submission_files =['swinL_384', 'swinL_224', 'swin_balanced', 'swin_anchor', 'yolo5l6', 'yolo5x6', 'effd4', 'effd5']
weights = [3, 3, 3, 3, 1, 1, 1, 1]

submission_df = [pd.read_csv(csvs[file]) for file in submission_files]

In [86]:
weights

[3, 3, 3, 3, 1, 1, 1, 1]

In [87]:
image_ids = submission_df[0]['image_id'].tolist()

In [88]:
annotation = '../dataset/test.json'
coco = COCO(annotation)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [89]:
prediction_strings = []
file_names = []
iou_thr = 0.6
cnt = 0

for i, image_id in enumerate(image_ids):
    prediction_string = ''
    boxes_list = []
    scores_list = []
    labels_list = []
    image_info = coco.loadImgs(i)[0]
    
    for df in submission_df:
        predict_string = df[df['image_id'] == image_id]['PredictionString'].tolist()[0]
        predict_list = str(predict_string).split()
        
        if len(predict_list)==0 or len(predict_list)==1:
            continue
            
        predict_list = np.reshape(predict_list, (-1, 6))
        box_list = []
        
        for box in predict_list[:, 2:6].tolist():
            box[0] = float(box[0]) / image_info['width']
            box[1] = float(box[1]) / image_info['height']
            box[2] = float(box[2]) / image_info['width']
            box[3] = float(box[3]) / image_info['height']
            box_list.append(box)
            
        boxes_list.append(box_list)
        scores_list.append(list(map(float, predict_list[:, 1].tolist())))
        labels_list.append(list(map(int, predict_list[:, 0].tolist())))
    
    if len(boxes_list):
        boxes, scores, labels = weighted_boxes_fusion(boxes_list, scores_list, labels_list, iou_thr=iou_thr, weights=weights)
        labels = labels.astype(np.int32)
        for box, score, label in zip(boxes, scores, labels):
            
            if score < 0.002:
                continue

#             w = box[3] * image_info['height'] - box[1] * image_info['height']
#             b = box[2] * image_info['width'] - box[0] * image_info['width']
            
#             if w * b < 300:
#                 cnt += 1
#                 continue

            prediction_string += str(label) + ' ' + str(round(score, 5)) + ' ' + str(round(box[0] * image_info['width'], 5)) + ' ' + str(round(box[1] * image_info['height'], 5)) + ' ' + str(round(box[2] * image_info['width'], 5)) + ' ' + str(round(box[3] * image_info['height'], 5)) + ' '
    
    prediction_strings.append(prediction_string)
    file_names.append(image_id)

/opt/conda/envs/mmdetection/lib/python3.8/site-packages/ensemble_boxes/ensemble_boxes_wbf.py:88: UserWarning: Zero area box skipped: [1.0, 0.2523115234375, 1.0, 1.0].
  warnings.warn("Zero area box skipped: {}.".format(box_part))
/opt/conda/envs/mmdetection/lib/python3.8/site-packages/ensemble_boxes/ensemble_boxes_wbf.py:88: UserWarning: Zero area box skipped: [1.0, 0.2510322265625, 1.0, 1.0].
  warnings.warn("Zero area box skipped: {}.".format(box_part))
/opt/conda/envs/mmdetection/lib/python3.8/site-packages/ensemble_boxes/ensemble_boxes_wbf.py:88: UserWarning: Zero area box skipped: [1.0, 0.0, 1.0, 0.8549990234375].
  warnings.warn("Zero area box skipped: {}.".format(box_part))
/opt/conda/envs/mmdetection/lib/python3.8/site-packages/ensemble_boxes/ensemble_boxes_wbf.py:85: UserWarning: Y2 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.
  warnings.warn('Y2 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.')
/opt/conda/envs/mmdetection/l

/opt/conda/envs/mmdetection/lib/python3.8/site-packages/ensemble_boxes/ensemble_boxes_wbf.py:88: UserWarning: Zero area box skipped: [0.0, 0.0, 0.0, 0.6872001953125].
  warnings.warn("Zero area box skipped: {}.".format(box_part))
/opt/conda/envs/mmdetection/lib/python3.8/site-packages/ensemble_boxes/ensemble_boxes_wbf.py:88: UserWarning: Zero area box skipped: [0.0, 1.0, 0.7643701171875, 1.0].
  warnings.warn("Zero area box skipped: {}.".format(box_part))
/opt/conda/envs/mmdetection/lib/python3.8/site-packages/ensemble_boxes/ensemble_boxes_wbf.py:88: UserWarning: Zero area box skipped: [0.00131640625, 1.0, 0.736169921875, 1.0].
  warnings.warn("Zero area box skipped: {}.".format(box_part))
/opt/conda/envs/mmdetection/lib/python3.8/site-packages/ensemble_boxes/ensemble_boxes_wbf.py:88: UserWarning: Zero area box skipped: [0.0, 1.0, 0.3766435546875, 1.0].
  warnings.warn("Zero area box skipped: {}.".format(box_part))
/opt/conda/envs/mmdetection/lib/python3.8/site-packages/ensemble_boxes/

/opt/conda/envs/mmdetection/lib/python3.8/site-packages/ensemble_boxes/ensemble_boxes_wbf.py:88: UserWarning: Zero area box skipped: [0.0, 0.0, 0.0, 0.5774169921875].
  warnings.warn("Zero area box skipped: {}.".format(box_part))
/opt/conda/envs/mmdetection/lib/python3.8/site-packages/ensemble_boxes/ensemble_boxes_wbf.py:88: UserWarning: Zero area box skipped: [0.0, 0.1044111328125, 0.0, 0.944974609375].
  warnings.warn("Zero area box skipped: {}.".format(box_part))
/opt/conda/envs/mmdetection/lib/python3.8/site-packages/ensemble_boxes/ensemble_boxes_wbf.py:88: UserWarning: Zero area box skipped: [1.0, 0.0, 1.0, 0.64055859375].
  warnings.warn("Zero area box skipped: {}.".format(box_part))
/opt/conda/envs/mmdetection/lib/python3.8/site-packages/ensemble_boxes/ensemble_boxes_wbf.py:88: UserWarning: Zero area box skipped: [0.0, 0.0, 0.0, 0.5814814453125].
  warnings.warn("Zero area box skipped: {}.".format(box_part))
/opt/conda/envs/mmdetection/lib/python3.8/site-packages/ensemble_boxes/

/opt/conda/envs/mmdetection/lib/python3.8/site-packages/ensemble_boxes/ensemble_boxes_wbf.py:88: UserWarning: Zero area box skipped: [0.0, 0.2345791015625, 0.0, 1.0].
  warnings.warn("Zero area box skipped: {}.".format(box_part))
/opt/conda/envs/mmdetection/lib/python3.8/site-packages/ensemble_boxes/ensemble_boxes_wbf.py:88: UserWarning: Zero area box skipped: [1.0, 0.0, 1.0, 0.453451171875].
  warnings.warn("Zero area box skipped: {}.".format(box_part))
/opt/conda/envs/mmdetection/lib/python3.8/site-packages/ensemble_boxes/ensemble_boxes_wbf.py:88: UserWarning: Zero area box skipped: [1.0, 0.3453935546875, 1.0, 1.0].
  warnings.warn("Zero area box skipped: {}.".format(box_part))
/opt/conda/envs/mmdetection/lib/python3.8/site-packages/ensemble_boxes/ensemble_boxes_wbf.py:88: UserWarning: Zero area box skipped: [1.0, 0.0, 1.0, 0.51041015625].
  warnings.warn("Zero area box skipped: {}.".format(box_part))
/opt/conda/envs/mmdetection/lib/python3.8/site-packages/ensemble_boxes/ensemble_box

/opt/conda/envs/mmdetection/lib/python3.8/site-packages/ensemble_boxes/ensemble_boxes_wbf.py:88: UserWarning: Zero area box skipped: [0.0, 0.0, 0.0, 0.631189453125].
  warnings.warn("Zero area box skipped: {}.".format(box_part))
/opt/conda/envs/mmdetection/lib/python3.8/site-packages/ensemble_boxes/ensemble_boxes_wbf.py:88: UserWarning: Zero area box skipped: [1.0, 0.0, 1.0, 0.6481669921875].
  warnings.warn("Zero area box skipped: {}.".format(box_part))
/opt/conda/envs/mmdetection/lib/python3.8/site-packages/ensemble_boxes/ensemble_boxes_wbf.py:88: UserWarning: Zero area box skipped: [0.0, 0.2023173828125, 0.0, 0.95391015625].
  warnings.warn("Zero area box skipped: {}.".format(box_part))
/opt/conda/envs/mmdetection/lib/python3.8/site-packages/ensemble_boxes/ensemble_boxes_wbf.py:88: UserWarning: Zero area box skipped: [1.0, 0.213341796875, 1.0, 0.995125].
  warnings.warn("Zero area box skipped: {}.".format(box_part))
/opt/conda/envs/mmdetection/lib/python3.8/site-packages/ensemble_bo

/opt/conda/envs/mmdetection/lib/python3.8/site-packages/ensemble_boxes/ensemble_boxes_wbf.py:88: UserWarning: Zero area box skipped: [1.0, 0.0, 1.0, 0.220333984375].
  warnings.warn("Zero area box skipped: {}.".format(box_part))
/opt/conda/envs/mmdetection/lib/python3.8/site-packages/ensemble_boxes/ensemble_boxes_wbf.py:88: UserWarning: Zero area box skipped: [0.0, 0.0, 0.0, 0.6459189453125].
  warnings.warn("Zero area box skipped: {}.".format(box_part))
/opt/conda/envs/mmdetection/lib/python3.8/site-packages/ensemble_boxes/ensemble_boxes_wbf.py:88: UserWarning: Zero area box skipped: [1.0, 0.0, 1.0, 0.704140625].
  warnings.warn("Zero area box skipped: {}.".format(box_part))
/opt/conda/envs/mmdetection/lib/python3.8/site-packages/ensemble_boxes/ensemble_boxes_wbf.py:88: UserWarning: Zero area box skipped: [1.0, 0.115181640625, 1.0, 0.9860341796875001].
  warnings.warn("Zero area box skipped: {}.".format(box_part))
/opt/conda/envs/mmdetection/lib/python3.8/site-packages/ensemble_boxes/

/opt/conda/envs/mmdetection/lib/python3.8/site-packages/ensemble_boxes/ensemble_boxes_wbf.py:88: UserWarning: Zero area box skipped: [1.0, 0.0, 1.0, 0.7318173828125].
  warnings.warn("Zero area box skipped: {}.".format(box_part))
/opt/conda/envs/mmdetection/lib/python3.8/site-packages/ensemble_boxes/ensemble_boxes_wbf.py:88: UserWarning: Zero area box skipped: [1.0, 0.49842578125, 1.0, 1.0].
  warnings.warn("Zero area box skipped: {}.".format(box_part))
/opt/conda/envs/mmdetection/lib/python3.8/site-packages/ensemble_boxes/ensemble_boxes_wbf.py:88: UserWarning: Zero area box skipped: [1.0, 0.0223525390625, 1.0, 0.8649755859375].
  warnings.warn("Zero area box skipped: {}.".format(box_part))
/opt/conda/envs/mmdetection/lib/python3.8/site-packages/ensemble_boxes/ensemble_boxes_wbf.py:88: UserWarning: Zero area box skipped: [1.0, 0.0, 1.0, 0.3624990234375].
  warnings.warn("Zero area box skipped: {}.".format(box_part))
/opt/conda/envs/mmdetection/lib/python3.8/site-packages/ensemble_boxes

In [9]:
cnt

2899

In [90]:
submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv('3weight.csv')
submission.head()

,PredictionString,image_id
0,7 0.99536 216.06578 51.06284 454.42389 474.310...,test/0000.jpg
1,6 0.8633 760.20288 663.61243 899.1272 990.1270...,test/0001.jpg
2,1 0.93144 296.98438 312.63791 1014.67749 748.8...,test/0002.jpg
3,9 0.84668 155.91902 261.61337 904.64886 814.24...,test/0003.jpg
4,1 0.89363 199.29022 253.1062 871.49438 776.646...,test/0004.jpg


In [198]:
df = pd.read_csv('1.csv')

In [235]:
import time
max_len = 0
cnt = 0
tot = 0
for i, x in df.iterrows():
    p = list(map(float, x['PredictionString'].split()[1::6]))
    w = np.array(list(map(float, x['PredictionString'].split()[4::6]))) - np.array(list(map(float, x['PredictionString'].split()[2::6])))
    h = np.array(list(map(float, x['PredictionString'].split()[5::6]))) - np.array(list(map(float, x['PredictionString'].split()[3::6])))
    
    for p_, w_, h_ in zip(p, w, h):
        tot += 1
        if w_*h_ <= 300:
            cnt += 1
            # if p_ > 0.1:
            #     print(p_, w_*h_)
(cnt*100)/tot

2.6369904813110177

In [215]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4871 entries, 0 to 4870
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   PredictionString  4871 non-null   object
 1   image_id          4871 non-null   object
dtypes: object(2)
memory usage: 76.2+ KB
